## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,84.2,79,40,10.36,scattered clouds
1,1,Mar Del Plata,AR,-38.0023,-57.5575,66.0,88,40,11.50,scattered clouds
2,2,Shelburne,CA,44.0787,-80.2041,37.0,100,99,24.92,snow
3,3,Ushuaia,AR,-54.8000,-68.3000,46.4,57,75,11.50,broken clouds
4,4,Riyadh,SA,24.6877,46.7219,62.6,22,0,5.75,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_hotel_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
#preferred_cities_df.head(10)
clean_hotel_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,84.20,79,40,10.36,scattered clouds
6,6,Ixtapa,MX,20.7000,-105.2000,75.20,73,75,11.50,broken clouds
7,7,Araguacu,BR,-12.9306,-49.8264,75.97,78,88,0.29,overcast clouds
15,15,Kapaa,US,22.0752,-159.3190,77.00,69,90,16.11,light rain
16,16,Atuona,PF,-9.8000,-139.0333,80.96,75,91,16.53,overcast clouds
18,18,Kabankalan,PH,9.9889,122.8122,82.08,73,98,3.18,overcast clouds
22,22,Avarua,CK,-21.2078,-159.7750,82.40,83,75,16.11,broken clouds
24,24,Saint-Philippe,RE,-21.3585,55.7679,75.20,78,16,5.75,few clouds
30,30,Georgetown,MY,5.4112,100.3354,80.01,94,20,5.35,light rain
35,35,Faanui,PF,-16.4833,-151.7500,82.76,72,45,20.07,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
clean_hotel_df.count()

City_ID                163
City                   163
Country                163
Lat                    163
Lng                    163
Max Temp               163
Humidity               163
Cloudiness             163
Wind Speed             163
Current Description    163
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
#hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
#hotel_df["Hotel Name"] = ""
#hotel_df.head(10)

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
new_hotel_df = clean_hotel_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
new_hotel_df["Hotel Name"] = ""
new_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,84.20,scattered clouds,-21.2000,-175.2000,
6,Ixtapa,MX,75.20,broken clouds,20.7000,-105.2000,
7,Araguacu,BR,75.97,overcast clouds,-12.9306,-49.8264,
15,Kapaa,US,77.00,light rain,22.0752,-159.3190,
16,Atuona,PF,80.96,overcast clouds,-9.8000,-139.0333,
18,Kabankalan,PH,82.08,overcast clouds,9.9889,122.8122,
22,Avarua,CK,82.40,broken clouds,-21.2078,-159.7750,
24,Saint-Philippe,RE,75.20,few clouds,-21.3585,55.7679,
30,Georgetown,MY,80.01,light rain,5.4112,100.3354,
35,Faanui,PF,82.76,scattered clouds,-16.4833,-151.7500,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in new_hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        new_hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
new_hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [18]:
# 7. Drop the rows where there is no Hotel Name.
clean_cities_df = new_hotel_df.dropna()
clean_cities_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,84.20,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
6,Ixtapa,MX,75.20,broken clouds,20.7000,-105.2000,Hotel Plaza Providencia
7,Araguacu,BR,75.97,overcast clouds,-12.9306,-49.8264,Hotel Tocantins
15,Kapaa,US,77.00,light rain,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
16,Atuona,PF,80.96,overcast clouds,-9.8000,-139.0333,Villa Enata
...,...,...,...,...,...,...,...
538,Unguia,CO,77.74,broken clouds,8.0434,-77.0945,Residencias El Golfo
541,Bowen,AU,82.40,broken clouds,-20.0167,148.2333,Castle Motor Lodge Motel
544,Hirara,JP,75.20,broken clouds,24.8000,125.2833,Hotel Atoll Emerald Miyakojima
552,Hambantota,LK,76.41,overcast clouds,6.1241,81.1185,Bungalow 63


In [19]:
# 8a. Create the output File (CSV)

output_data_file = "Weather_Database/Weather_Hotel_Database.csv"

# 8b. Export the City_Data into a csv
clean_cities_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
#info_box_template = """

#"""

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_cities_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_cities_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
#fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(marker_layer)
# 11b. Display the figure

locations = clean_cities_df[["Lat", "Lng"]]
max_temp = clean_cities_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))